# 1. Data upload

In [1]:
import pandas as pd

In [2]:
#загружаем 1-ый датасет, удаляем дубликаты и выводим размерность
df_h = pd.read_csv('ga_hits.csv', sep = ',')
df_h.drop_duplicates()
df_h.shape

(15726470, 11)

In [3]:
#аналогично для 2-го датасета
df_s = pd.read_csv('ga_sessions.csv', sep = ',')
df_s.drop_duplicates()
df_s.shape

C:\Users\user\AppData\Local\Temp\ipykernel_2032\2022408432.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_s = pd.read_csv('ga_sessions.csv', sep = ',')


(1860042, 18)

In [4]:
#соединяем 2 датасета по столбцу (session_id), удаляем дубликаты смотрим информацию о получившемся датасете
df_old = pd.merge(df_h, df_s, how = 'inner', on = 'session_id')
df_old.drop_duplicates()
print(df_old.info(), df_old.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15685219 entries, 0 to 15685218
Data columns (total 28 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   hit_date                  object 
 2   hit_time                  float64
 3   hit_number                int64  
 4   hit_type                  object 
 5   hit_referer               object 
 6   hit_page_path             object 
 7   event_category            object 
 8   event_action              object 
 9   event_label               object 
 10  event_value               float64
 11  client_id                 object 
 12  visit_date                object 
 13  visit_time                object 
 14  visit_number              int64  
 15  utm_source                object 
 16  utm_medium                object 
 17  utm_campaign              object 
 18  utm_adcontent             object 
 19  utm_keyword               object 
 20  device_category       

# 2. Data preparation

In [5]:
#оцениваем проценты пропущенных значений для прояснения логики дальнейших действий
missing_values_percent = ( (df_old.isna().sum() / len(df_old))* 100 ).sort_values(ascending=False)
missing_values_percent 

event_value                 100.000000
device_model                 99.220368
utm_keyword                  58.684721
hit_time                     58.400224
device_os                    58.388614
hit_referer                  39.753975
device_brand                 25.156646
event_label                  23.765572
utm_adcontent                18.057880
utm_campaign                 14.018759
utm_source                    0.004463
session_id                    0.000000
geo_country                   0.000000
device_browser                0.000000
device_screen_resolution      0.000000
device_category               0.000000
visit_number                  0.000000
utm_medium                    0.000000
hit_date                      0.000000
visit_time                    0.000000
visit_date                    0.000000
client_id                     0.000000
event_action                  0.000000
event_category                0.000000
hit_page_path                 0.000000
hit_type                 

In [29]:
# исходя из методических указаний к работе, удаляем некоторые столбцы. Также удаляем device_model (99% значений пропущено)
df = df_old.drop(columns = ['event_category', 'event_label', 'event_value', 'device_model', 'hit_time', 'hit_referer', 'visit_number', 'visit_time', 'visit_date', 'client_id', 'hit_date',
'hit_page_path', 'hit_type', 'hit_number', 'session_id'], axis = 1)
df.shape

(15685219, 13)

In [30]:
# из event_action получаем target, целевую переменную. Затем исходную колонку удаляем - не нужна.
def f(df):
    if df['event_action'] in ['sub_car_claim_click', 'sub_car_claim_submit_click','sub_open_dialog_click', 'sub_custom_question_submit_click', 'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success','sub_car_request_submit_click']:
        val = 1
    else:
        val = 0
    return val

df['target'] = df.apply (f, axis=1)
df.drop(columns = 'event_action', inplace = True)
#удаляем дубликаты
df = df.drop_duplicates()

In [31]:
# удаляем строки, где utm_source и utm_medium не заполнены, 
# так как относительно размеров датасета их немного и пропущены они, как правило, в одних и тех же строках
df = df[df.utm_medium.notna()]
df = df[df.utm_source.notna()]
# в нижеуказанных колонках заполняем пропуски значением other
df.utm_campaign = df.utm_campaign.fillna('other')
df.utm_adcontent = df.utm_adcontent.fillna('other')
df.device_os = df.device_os.fillna('other')
df.utm_keyword = df.utm_keyword.fillna('other')
df.device_brand = df.device_brand.fillna('other')

In [32]:
#в силу ограниченных возможностей для вычисления сокращаем количество значений в столбцах (где это представляется возможным)
df['device_brand'] = df['device_brand'].apply(lambda x: x if x == 'Apple' or x== 'Samsung' or x == 'Xiaomi' or x == 'Huawei' else 'other')
df['device_os'] = df['device_os'].apply(lambda x: x if x == 'Android' or x == 'iOS' or x == 'Windows' or x == 'Macintosh' or x == 'Linux' else 'other')
df['device_browser'] = df['device_browser'].apply(lambda x: x if x == 'Safari' or x == 'YaBrowser' or x == 'Safari (in-app)' or x == 'Android Webview' or x == 'Samsung Internet' or x == 'Firefox' or x == 'Edge' else 'other')
df['geo_country'] = df['geo_country'].apply(lambda x: x if x == 'Russia' or x == 'Ukraine' or x == 'United States' or x == 'Belarus' or x == 'Germany' else 'other')
df['geo_city'] = df['geo_city'].apply(lambda x: x if x == 'Moscow' or x == 'Saint Petersburg' or x == 'Yekaterinburg' or x == 'Krasnodar' else 'other')
df['device_screen_resolution'] = df['device_screen_resolution'].apply(lambda x: 'low' if len(x) <= 7   else ('medium' if len(x) == 8 else 'high' ))

In [33]:
#снова смотрим процент пропущенных значений - он равен 0
missing_values_percent = ( (df.isna().sum() / len(df))* 100 ).sort_values(ascending=False)
missing_values_percent

utm_source                  0.0
utm_medium                  0.0
utm_campaign                0.0
utm_adcontent               0.0
utm_keyword                 0.0
device_category             0.0
device_os                   0.0
device_brand                0.0
device_screen_resolution    0.0
device_browser              0.0
geo_country                 0.0
geo_city                    0.0
target                      0.0
dtype: float64

In [34]:
#все колонки, кроме таргета, приводим к типу str
df = df.astype('str')
df.target = df.target.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417600 entries, 0 to 15685207
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   utm_source                417600 non-null  object
 1   utm_medium                417600 non-null  object
 2   utm_campaign              417600 non-null  object
 3   utm_adcontent             417600 non-null  object
 4   utm_keyword               417600 non-null  object
 5   device_category           417600 non-null  object
 6   device_os                 417600 non-null  object
 7   device_brand              417600 non-null  object
 8   device_screen_resolution  417600 non-null  object
 9   device_browser            417600 non-null  object
 10  geo_country               417600 non-null  object
 11  geo_city                  417600 non-null  object
 12  target                    417600 non-null  int32 
dtypes: int32(1), object(12)
memory usage: 43.0+ MB


In [35]:
# сохраняем получившийся датасет. Следующие этапы будут реализованы в виде пайплайна при помощи PyCharm
df.to_csv('df.csv', index = False)

In [36]:
df.shape

(417600, 13)

In [47]:
df.head()

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target
0,kjsLglQLzykiRbcDiGcD,cpc,other,other,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,low,other,Russia,Saint Petersburg,0
38,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,low,other,Russia,other,0
83,BHcvLfOaCWvWTykYqHVe,cpc,other,other,JajANoFxoqXfKRNBUhzx,desktop,Windows,other,high,other,Russia,Moscow,0
119,BHcvLfOaCWvWTykYqHVe,cpc,other,other,zXjuoqEaPElBnwpSHMpx,mobile,Android,Samsung,low,other,Russia,other,0
157,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,other,medium,other,other,other,0
